In [1]:
import warnings
import numpy as np
from tensorflow.keras.layers import Input, Dense, Lambda, Concatenate, Conv2D, Conv2DTranspose, Flatten, Reshape, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.applications import ResNet50
import cv2
import matplotlib.pyplot as plt
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

warnings.filterwarnings('ignore')

2023-07-25 08:01:07.677018: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 08:01:08.166991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import h5py
with h5py.File("/home/danielaguirre/Documents/university/masteringml/MML-project/data/processed_data.h5", "r") as f:
    imgs_train, labels_train = np.array(f["X_train"]), np.array(f["y_train"])
    imgs_test, labels_test = np.array(f["X_test"]), np.array(f["y_test"])

imgs_train2 = []
imgs_test2 = []
for img in imgs_train:
    new_img = cv2.resize(img, (128,128))
    new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
    imgs_train2.append(new_img)
for img in imgs_test:
    new_img = cv2.resize(img, (128,128))
    new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
    imgs_test2.append(new_img)
imgs_train = np.array(imgs_train2)[:, :, :, np.newaxis]
imgs_test = np.array(imgs_test2)[:, :, :, np.newaxis]
imgs_train2 = (imgs_train-127.5)/127.5
imgs_test2 = (imgs_test-127.5)/127.5

In [3]:
batch_size = 128 # batch size
latent_dim = 256 # latent space size
optim = Adam(lr=0.0001)
shape_img = imgs_train.shape[1:]
n_cat = labels_train.shape[1]
n_epoch = 300

In [4]:
resnet = ResNet50(weights=None, include_top=False, input_shape=(128,128,1))

2023-07-25 08:01:09.351826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 08:01:09.363769: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 08:01:09.363898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Instructions for updating:
Colocations handled automatically by placer.


2023-07-25 08:01:09.370613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 08:01:09.370719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 08:01:09.370805: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
enc_input_image = Input(shape=(shape_img), name="input_image")
enc_input_label = Input(shape=(n_cat), name="input_label")

enc = resnet(enc_input_image)
# convolutionals block
enc_conv1 = Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")(enc) # 32x32
enc_conv2 = Conv2D(filters=64, kernel_size=3, padding="same", activation='relu')(enc_conv1) #  skip-connection 16x16
drop_out = Dropout(rate=0.3)(enc_conv2)
enc_conv3 = Conv2D(filters=128, kernel_size=3, padding="same", activation='relu')(drop_out) # 8x8
drop_out = Dropout(rate=0.3)(enc_conv3)
enc_conv4 = Conv2D(filters=256, kernel_size=3, padding="same", activation='relu')(drop_out)  #  skip-connection # 16x16
enc_conv5 = Conv2D(filters=512, kernel_size=3, padding="same", activation='relu')(enc_conv4)  #  skip-connection # 16x16

# concat
flattened = Flatten()(enc_conv5)
enc_concat = Concatenate()([flattened, enc_input_label])

encoder_hidden = Dense(1048, name="enc_hidden_layer")(enc_concat)
drop_out = Dropout(rate=0.3)(encoder_hidden)
encoder_hidden = Dense(512, name="enc_hidden_layer2")(drop_out)

mu = Dense(latent_dim, activation='linear', name="mu")(encoder_hidden)
l_sigma = Dense(latent_dim, activation='linear', name="l_sigma")(encoder_hidden)
def sample_z(args):
    mu, l_sigma = args
    eps = K.random_normal(shape=(latent_dim, ), mean=0., stddev=1.) # shape antes => (batch_size, latent_dim)
    return mu + K.exp(l_sigma / 2) * eps
z = Lambda(sample_z, output_shape = (latent_dim, ), name="latent_vector")([mu, l_sigma]) # output encoder

encoder = Model([enc_input_image, enc_input_label], z, name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_image (InputLayer)    [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 resnet50 (Functional)       (None, 4, 4, 2048)           2358144   ['input_image[0][0]']         
                                                          0                                       
                                                                                                  
 conv2d (Conv2D)             (None, 4, 4, 32)             589856    ['resnet50[0][0]']            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 4, 4, 64)             18496     ['conv2d[0][0]']        

In [6]:
dec_inp_latent_vector = Input(shape=(latent_dim), name="input_latent_vector")
decoder_inp_label = Input(shape=(n_cat), name = "input_label")
dec_concat = Concatenate(name="decoder_concat")([dec_inp_latent_vector, decoder_inp_label])

decoder_hidden = Dense(512, activation="relu", name="hidden_layer1")(dec_concat)
decoder_hidden = Dense(512, activation="relu", name="hidden_layer2")(decoder_hidden)
decoder_hidden = Dropout(0.3)(decoder_hidden)
decoder_hidden = Dense(512, activation="relu", name="hidden_layer3")(decoder_hidden)
decoder_hidden = Dense(512, activation="relu", name="hidden_layer4")(decoder_hidden)
decoder_hidden = Dropout(0.3)(decoder_hidden)
decoder_hidden = Dense(512, activation="relu", name="hidden_layer5")(decoder_hidden)
decoder_hidden = Dense(1048, activation="relu", name="hidden_layer6")(decoder_hidden)
decoder_hidden = Dropout(rate=0.3)(decoder_hidden)
decoder_hidden = Dense(16*16*32, activation="relu", name="hidden_layer7")(decoder_hidden)
reshaped = Reshape(target_shape=(16,16,32))(decoder_hidden)

# convolutionals block
dec_conv1 = Conv2D(filters=512, kernel_size=3, padding="same", activation='relu')(reshaped)  #  skip-connection # 16x16
dec_conv1 = Conv2DTranspose(filters=256, kernel_size=3, activation="relu", padding='same')(dec_conv1)
dec_conv1 = Dropout(rate=0.3)(dec_conv1)
dec_conv2 = Conv2DTranspose(filters=128, kernel_size=3, activation="relu", strides=2, padding='same')(dec_conv1)
dec_conv3 = Conv2DTranspose(filters=64, kernel_size=3, activation="relu", strides=2, padding='same')(dec_conv2)
dec_conv3 = Dropout(rate=0.3)(dec_conv3)
dec_conv4 = Conv2DTranspose(filters=32, kernel_size=3, activation="relu", strides=2, padding='same')(dec_conv3)

dec_output_img = Conv2DTranspose(filters=1, kernel_size=3, padding="same", activation="sigmoid")(dec_conv4)
dec_output_img = Flatten()(dec_output_img)
decoder = Model([dec_inp_latent_vector, decoder_inp_label], dec_output_img, name="decoder")
decoder.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_latent_vector (Input  [(None, 256)]                0         []                            
 Layer)                                                                                           
                                                                                                  
 input_label (InputLayer)    [(None, 4)]                  0         []                            
                                                                                                  
 decoder_concat (Concatenat  (None, 260)                  0         ['input_latent_vector[0][0]', 
 e)                                                                  'input_label[0][0]']         
                                                                                            

In [7]:
latent_vector = encoder(inputs=[enc_input_image, enc_input_label]) # z
output_img = decoder(inputs=[latent_vector, enc_input_label])
CVAE = Model(inputs=[enc_input_image, enc_input_label], outputs=output_img, name="CVAE")
CVAE.summary()

Model: "CVAE"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_image (InputLayer)    [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 input_label (InputLayer)    [(None, 4)]                  0         []                            
                                                                                                  
 encoder (Functional)        (None, 256)                  3512917   ['input_image[0][0]',         
                                                          6          'input_label[0][0]']         
                                                                                                  
 decoder (Functional)        (None, 16384)                1203074   ['encoder[0][0]',          

In [8]:
def vae_loss(y_true, y_pred):
    recon = recon_loss(y_true, y_pred)
    kl = KL_loss(y_true, y_pred)
    return recon + kl

def KL_loss(y_true, y_pred): # it doesnt use y_true and y_pred but the parameters are necessary for compiling
     # importante se usan las capas de mu y sigma del encoder
    return(0.5 * K.sum(K.exp(l_sigma) + K.square(mu) - 1. - l_sigma, axis=1))

def recon_loss(y_true, y_pred):
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)

In [9]:
CVAE.compile(optimizer=optim, loss=vae_loss, metrics=[KL_loss, recon_loss])

In [10]:
# the output of the decoder is a flattened img,
# so we need to flatten the true values (input images from (28,28,1) => 784)
y_train = imgs_train.reshape(imgs_train.shape[0], -1)
y_test = imgs_test.reshape(imgs_test.shape[0], -1)
print(y_train.shape)

(2937, 16384)


In [ ]:
cvae_hist = CVAE.fit([imgs_train2, labels_train], y_train/255, verbose = 1, batch_size=batch_size, epochs=n_epoch,
                      validation_data = ([imgs_test2, labels_test], y_test/255))

Train on 2937 samples, validate on 327 samples


2023-07-25 08:01:18.947466: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 08:01:18.947613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-25 08:01:18.947758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/300


2023-07-25 08:01:25.070793: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indropout/cond/then/_2968/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-07-25 08:01:28.799917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-07-25 08:01:29.510855: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-25 08:01:29.511269: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-25 08:01:29.511280: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:109] Couldn't get ptxas version : FAILED_PRECONDITION: Couldn't get ptxas/nvlink version string: INTERNAL: Couldn't invoke ptxas --version
2023-07-25 08:01:29.511696: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn 

2937/2937 [==============================] - ETA: 0s - loss: 10454.7431 - KL_loss: 5.4056 - recon_loss: 10449.3379

2023-07-25 08:01:44.977670: W tensorflow/c/c_api.cc:304] Operation '{name:'loss/mul' id:8414 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/decoder_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-07-25 08:01:46.066110: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape indropout/cond/then/_2968/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2937/2937 [==============================] - 24s 8ms/sample - loss: 10454.7431 - KL_loss: 5.4056 - recon_loss: 10449.3379 - val_loss: 8643.2126 - val_KL_loss: 0.0989 - val_recon_loss: 8643.1133
Epoch 2/300
2937/2937 [==============================] - 8s 3ms/sample - loss: 7961.3712 - KL_loss: 0.7805 - recon_loss: 7960.5918 - val_loss: 7594.7330 - val_KL_loss: 0.3510 - val_recon_loss: 7594.3823
Epoch 3/300
2937/2937 [==============================] - 8s 3ms/sample - loss: 7258.7036 - KL_loss: 39.1164 - recon_loss: 7219.5874 - val_loss: 7439.0857 - val_KL_loss: 2.5339 - val_recon_loss: 7436.5518
Epoch 4/300
2937/2937 [==============================] - 8s 3ms/sample - loss: 7138.6866 - KL_loss: 38.7641 - recon_loss: 7099.9224 - val_loss: 7445.3733 - val_KL_loss: 4.1896 - val_recon_loss: 7441.1836
Epoch 5/300
2937/2937 [==============================] - 8s 3ms/sample - loss: 7014.4213 - KL_loss: 32.0380 - recon_loss: 6982.3833 - val_loss: 7258.3702 - val_KL_loss: 8.7859 - val_recon_loss: 7

In [ ]:
import matplotlib.pyplot as plt

plt.plot(cvae_hist.history['loss'])
plt.plot(cvae_hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
z = np.zeros(shape=(1, latent_dim))  # latent vector
z = np.random.normal(size=(1, latent_dim))
label = np.array([[0, 1, 0, 0]])  # label in one hot encoding
plt.figure(figsize=(3, 3))
plt.imshow(decoder.predict([z, label]).reshape(128, 128), cmap=plt.cm.gray)
plt.show()